In [ ]:
# clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
!git reset --hard fbe67e465375231474a2ad80a4389efc77ecff99

fatal: destination path 'yolov5' already exists and is not an empty directory.
/content/yolov5
HEAD is now at fbe67e46 Fix `OMP_NUM_THREADS=1` for macOS (#8624)


In [ ]:
# install dependencies as necessary
!pip install -qr requirements.txt  # install dependencies (ignore errors)
import torch

from IPython.display import Image, clear_output  # to display images
from utils.downloads import attempt_download  # to download models/datasets

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Setup complete. Using torch 2.3.0+cu121 CPU


In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="wLcs5jy21EsrfRws42Vo")
project = rf.workspace("driver-wlf6m").project("driverv1")
version = project.version(3)
dataset = version.download("yolov5")


loading Roboflow workspace...
loading Roboflow project...


In [ ]:
import os
import shutil
import random


In [ ]:

# Function to split the dataset into train, validation, and test
def split_dataset(dataset_dir, output_dir, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15):
    assert train_ratio + val_ratio + test_ratio == 1.0, "Ratios must sum to 1.0"

    # Create train, validation, and test directories
    train_dir = os.path.join(output_dir, "train")
    val_dir = os.path.join(output_dir, "val")
    test_dir = os.path.join(output_dir, "test")

    for dir in [train_dir, val_dir, test_dir]:
        if not os.path.exists(dir):
            os.makedirs(dir)

    # Split dataset into train, validation, and test
    for class_name in os.listdir(dataset_dir):
        class_dir = os.path.join(dataset_dir, class_name)
        if os.path.isdir(class_dir):
            files = os.listdir(class_dir)
            random.shuffle(files)
            num_files = len(files)

            train_end = int(train_ratio * num_files)
            val_end = int((train_ratio + val_ratio) * num_files)

            # Move files to train, validation, and test directories
            for i, file in enumerate(files):
                src_file = os.path.join(class_dir, file)
                if i < train_end:
                    dst_dir = train_dir
                elif i < val_end:
                    dst_dir = val_dir
                else:
                    dst_dir = test_dir

                if not os.path.exists(os.path.join(dst_dir, class_name)):
                    os.makedirs(os.path.join(dst_dir, class_name))

                if os.path.isfile(src_file):
                    dst_file = os.path.join(dst_dir, class_name, file)
                    shutil.copy(src_file, dst_file)


In [ ]:
# Fungsi untuk menyamakan jumlah sampel setiap kelas
def balance_classes(dataset_dir, target_count):
    for class_name in os.listdir(dataset_dir):
        class_dir = os.path.join(dataset_dir, class_name)
        if os.path.isdir(class_dir):
            files = os.listdir(class_dir)
            num_files = len(files)
            if num_files > target_count:
                excess_files = files[target_count:]
                for file in excess_files:
                    src_file = os.path.join(class_dir, file)
                    dst_dir = os.path.join(dataset_dir, file.split('.')[0])  # Nama folder sesuai dengan nama file
                    dst_file = os.path.join(dst_dir, file)
                    if not os.path.exists(dst_dir):
                        os.makedirs(dst_dir)
                    shutil.move(src_file, dst_file)

# Mengatur direktori dataset
dataset_dir = "/content/yolov5/DriverV1-3"
output_dir = "/content/Processed_Dataset"



In [ ]:

# Memisahkan dataset menjadi train, validation, dan test
split_dataset(dataset_dir, output_dir)

In [ ]:
# Menyamakan jumlah sampel setiap kelas
target_count = 1000
for split in ["train", "val", "test"]:
    balance_classes(os.path.join(output_dir, split), target_count)

In [ ]:
from google.colab import files

# Mengompres folder output_dir menjadi file zip
shutil.make_archive("/content/Processed_Dataset", 'zip', output_dir)

# Mengunduh file zip
files.download("/content/Processed_Dataset.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>